# DC Extrapolation for Time Domain 

Extrapolates the low frequency points needed for time-domain transformations, when measurement doesn't include DC.

Example:

In [ ]:
import skrf as rf
from skrf.media import Coaxial
import matplotlib.pyplot as plt
from skrf.plotting import stylely
stylely()
%matplotlib inline

In [ ]:
freq = rf.Frequency(0.11, 110, 1001)
coax1mm = Coaxial(freq, z0=50, Dint=0.44e-3, Dout=1.0e-3, sigma=1e20)

X = coax1mm.line(10, unit='mm', z0=50, name='X', embed=True)
Y = coax1mm.line(80, unit='mm', z0=75, name='Y', embed=True)
dut = X**Y**X

dut_dc = dut.extrapolate_to_dc(dc_sparam=[[0,1],[1,0]])

In [ ]:
plt.figure()
plt.title('Step')
t, y = dut.s11.step_response(pad=2000)
t2, y2 = dut_dc.s11.step_response(pad=2000)
plt.plot(t*1e9, y, label='Original')
plt.plot(t2*1e9, y2, label='Extrapolated')
plt.legend()
plt.xlabel('Time (ns)')

In [ ]:
plt.figure()
plt.title('Impulse')
t, y = dut.s11.impulse_response(pad=2000)
t2, y2 = dut_dc.s11.impulse_response(pad=2000)
plt.plot(t*1e9, y, label='Original')
plt.plot(t2*1e9, y2, label='Extrapolated')
plt.legend()
plt.xlabel('Time (ns)')